# Applied Energy Final Figures
*by Fletcher Passow*

August 2025

## Purpose and scope

This notebook assembles the final figures and summary tables for the Applied Energy paper using results produced by the Kedro pipeline.

It:
- Loads partitioned results from the Kedro Data Catalog.
- Normalizes and groups scenario outputs (sharing, valet, vehicles per fleet, vocation pairs).
- Constructs derived metrics (LCOC, utilization, capacity per vehicle).
- Builds contrasts against consistent baselines to analyze strategy effects.
- Produces publication-ready plots saved to the reporting directory.

## Load `kedro` environment

In [ ]:
%reload_ext kedro.ipython

### About the Kedro IPython extension

`%reload_ext kedro.ipython` enables Kedro-specific IPython magics and integrates the project context and Data Catalog into this notebook session. This lets us load datasets via `catalog.load(...)` and reuse project configuration without manual wiring.

## Load modules

In [ ]:
import pandas as pd
import re
from copy import deepcopy
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.markers import MarkerStyle
from matplotlib.ticker import PercentFormatter, FuncFormatter
from typing import Tuple
from matplotlib.axes import Axes

from e_depot_strategies.pipelines.summarize.nodes import collate_scenario_partitions

In [ ]:
SMALL_SCENARIO_SET = "quads"
BIG_SCENARIO_SET = "quads_big"

In [ ]:
def set_reporting_groups(df: pd.DataFrame) -> pd.DataFrame:
    """Process the partition keys and counterfactuals into grouping columns."""
    df = df.reset_index()
    df[["voc_category_i", "voc_category_j"]] = df["voc_set"].str.split(
        "__", expand=True
    )
    df["task_id"] = (
        df["task_id"].str.extract(r"(?<=task_)(\d+)").astype(int)
    )
    df["veh_scen_id"] = (
        df["veh_set"].str.extract(r"(?<=veh_scen_)(\d+)").astype(int)
    )
    df["Vehicles per Fleet"] = 5
    df.loc[df["run_name"]==BIG_SCENARIO_SET, "Vehicles per Fleet"] = 15

    # Back to normal flow
    df = df.drop(columns=["voc_set", "scen_id", "veh_set"])
    df = df.set_index(["Vehicles per Fleet", "voc_category_i", "voc_category_j", "valet_set", "veh_scen_id", "task_id"])

    # Set shared-private dichotomy
    df["shar_priv"] = "shared"
    is_private = df["fleet_id"] != "shared"
    df.loc[is_private, "shar_priv"] = "private"
    return df


def eliminate_infeasible_scenarios(
    summs: pd.DataFrame, profs: pd.DataFrame
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Eliminate from summary the scenarios which had infeasible optimizations."""
    broken_rows = summs.loc[summs["problem_status"] == "infeasible_or_unbounded"]
    broken_scens = list(broken_rows.index.get_level_values("task_id").unique())
    summs = summs.drop(index=broken_scens, level="task_id")
    profs = profs.drop(index=broken_scens, level="task_id")
    return (summs, profs)

In [ ]:
def build_pairwise_array(ser:pd.Series, levels:Tuple[str, str]) -> pd.DataFrame:
    """Build a pairwise array using two index levels of a series."""
    pairs = ser.to_frame()
    pairs_opp = pairs.copy()
    pairs_opp.index.names = list(reversed(pairs.index.names))
    pairs_opp = pairs_opp.swaplevel(levels[0], levels[1])
    pairs = pd.concat([pairs, pairs_opp])
    pairs = pairs[ser.name]
    pairs = pairs.to_frame().reset_index()
    pairs = pairs.drop_duplicates([levels[1], ser.name]).set_index(list(levels)).sort_index()
    pairs = pairs.unstack(levels[1])
    pairs.columns = pairs.columns.droplevel(0)
    pairs.sort_index()
    return pairs

In [ ]:
def build_strategy_labels(valet_col:pd.Series, share_col:pd.Series, sep:str=", ") -> pd.Series:
    inter = valet_col.str.replace("_", " ").str.title() + sep + share_col.str.title()
    inter = pd.Categorical(inter, ordered=True)
    return inter

In [ ]:
def get_cbar_pct_labels(x, pos) -> str:
    """Built colorbar percentage labels."""
    xrep = int(x)
    if xrep > 0:
        prefix = "+"
    elif xrep < 0:
        prefix = "-"
    else:
        prefix = ""
    return prefix + str(xrep) + "%"

### Helper utilities

- `set_reporting_groups`: reshapes partition keys into analysis-friendly indices, sets shared vs private, parses task/vehicle scenario IDs, and standardizes "Vehicles per Fleet" (5 or 15).
- `eliminate_infeasible_scenarios`: drops scenarios whose optimization failed (infeasible or unbounded).
- `build_pairwise_array`: constructs symmetric pivoted matrices for vocation-pair summaries.
- `build_strategy_labels`: consistent, compact strategy labels combining valet and sharing settings.
- `get_cbar_pct_labels`: human-readable colorbar labels for percent-difference heatmaps.

## Set global plotting parameters

In [ ]:
from matplotlib import rc
sns.set_context("paper", font_scale=1.5)
sns.set_style("whitegrid")


#rc('font',**{'family':'serif','serif':['Helvetica']})
default_fig_dims_inches = (6.5, 4)

### Plotting defaults

Figures target paper scale (`sns.set_context('paper')`) with a clean `whitegrid` style and a compact default size. All plots are designed to:
- Compare strategies at-a-glance.
- Use consistent color palettes and axis formats (percent and currency).
- Save reproducibly for downstream manuscript use.

## Load data

In [ ]:
prof_parts = catalog.load("depot_profiles_part")
summ_parts = catalog.load("depot_summaries_part")
veh_parts = catalog.load("selected_vehicle_days_part")

In [ ]:
collate_params = catalog.load("params:summarize_partitions")
contrast_params = catalog.load("params:contrast")
collate_params.update({"include_partitions": [SMALL_SCENARIO_SET, BIG_SCENARIO_SET]})
contrast_params.update({"group_cols": ["Vehicles per Fleet", "voc_category_i", "voc_category_j", "veh_scen_id", "task_id", "shar_priv", "valet_set"]})
collate_params

In [ ]:
profs_df = collate_scenario_partitions(prof_parts, collate_params)
profs_df = set_reporting_groups(profs_df)
profs_df

In [ ]:
vehs_df = collate_scenario_partitions(veh_parts, collate_params)
vehs_df = set_reporting_groups(vehs_df)
vehs_df

In [ ]:
summs_df = collate_scenario_partitions(summ_parts, collate_params)
summs_df = set_reporting_groups(summs_df)
summs_df

In [ ]:
broken_scens = summs_df.loc[summs_df["n_valet_shifts"].isna(), :]
broken_tasks = broken_scens.index.get_level_values("task_id").unique()
broken_tasks

### Data and parameters

- Datasets loaded: depot profiles (`depot_profiles_part`), depot summaries (`depot_summaries_part`), and selected vehicle-days (`selected_vehicle_days_part`).
- Partitions included: `quads` and `quads_big` (see `collate_params`).
- Grouping dimensions for contrasts: `Vehicles per Fleet`, `voc_category_i/j`, `veh_scen_id`, `task_id`, `shar_priv`, and `valet_set`.
- We identify tasks with missing valet outputs (e.g., `n_valet_shifts`) for awareness; these can be excluded to avoid bias in contrast calculations.

## Explore valet presence

In [ ]:
valets_df = summs_df.loc[:, ["valets_per_shift"]]
valets_df["clean"] = valets_df["valets_per_shift"].str.replace("[", "")
valets_df["clean"] = valets_df["clean"].str.replace("]", "")
valets_df["clean"] = valets_df["clean"].str.replace(" ", "")
valets_df["list"] = valets_df["clean"].str.split(".")
for i in range(3):
    valets_df[f"valets_shift_{i}"] = valets_df["list"].transform(lambda s: float(s[i]))
valets_df = valets_df.drop(columns=["valets_per_shift", "clean", "list"])
valets_df = valets_df.query("valet_set =='with_valet'")
valets_df.sum(axis=0) / valets_df.shape[0]

### Valet presence sanity check

We parse and summarize `valets_per_shift` to confirm valet staffing is present and consistent in `with_valet` runs. This helps explain capacity and cost shifts where valet availability is a driver.

## Calculate contrasts

In [ ]:
cost_cols = list(filter(lambda x: re.compile(r"^cost_").match(x), summs_df.columns))
summs_df["total_cost_dollars"] = summs_df[cost_cols].sum(axis=1)

precursor = summs_df.groupby(contrast_params["group_cols"]).agg("sum")
precursor = precursor.select_dtypes(exclude=["object"])

# Calculate derived measurements
precursor["potential_energy_kwh"] = precursor["depot_capacity_kw"] * 24
precursor["utilization"] = (
    precursor["energy_delivered_kwh"] / precursor["potential_energy_kwh"]
)
precursor["cost_dollars_per_kwh"] = (
    precursor["total_cost_dollars"] / precursor["energy_delivered_kwh"]
)
precursor["capacity_kw_per_veh"] = precursor["depot_capacity_kw"] / precursor["n_vehicles"]
precursor["cost_upfront_per_kwh"] = precursor["cost_plugs_dollars"] / precursor["energy_delivered_kwh"]
precursor["cost_continue_per_kwh"] = precursor["cost_dollars_per_kwh"] - precursor["cost_upfront_per_kwh"]

precursor = precursor.droplevel("task_id", "index")
precursor = precursor.sort_index()
precursor = precursor.melt(ignore_index=False)
precursor = precursor.set_index("variable", append=True)
precursor = precursor.unstack(["Vehicles per Fleet", "shar_priv", "valet_set"])
precursor = precursor.droplevel(0, "columns")

idx_df = precursor.columns.to_frame()
idx_df["val_type"] = "orig"
idx_df = idx_df[["val_type", "Vehicles per Fleet", "shar_priv", "valet_set"]]
precursor.columns = pd.MultiIndex.from_frame(idx_df)

### Derived metrics and contrast setup

We compute:
- `potential_energy_kwh = depot_capacity_kw × 24`,
- `utilization = energy_delivered_kwh / potential_energy_kwh`,
- `cost_dollars_per_kwh = total_cost_dollars / energy_delivered_kwh`,
- `capacity_kw_per_veh = depot_capacity_kw / n_vehicles`,
- Split levelized cost into `cost_upfront_per_kwh` (plugs) and `cost_continue_per_kwh` (energy + demand + valet).

We then reindex values so each metric can be contrasted relative to baselines later (e.g., Private, No Valet).

## Attempt tabular summary

Build the main table using "Private, No Valet" as the baseline.

In [ ]:
contrast = deepcopy(precursor)
for val_type, n_vehs, share, valet in contrast.columns:
    contrast[("diff", n_vehs, share, valet)] = contrast[("orig", n_vehs, share, valet)] - contrast[("orig", n_vehs, "private", "no_valet")]
    contrast[("pct_diff", n_vehs, share, valet)] = contrast[("diff", n_vehs, share, valet)] / contrast[("orig", n_vehs, "private", "no_valet")] * 100
    contrast.loc[contrast[("diff", n_vehs, share, valet)] == 0, ("pct_diff", n_vehs, share, valet)] = 0

contrast = contrast.unstack("variable")
contrast

In [ ]:
keep_cols = ["cost_dollars_per_kwh", "capacity_kw_per_veh", "cost_upfront_per_kwh", "cost_continue_per_kwh"]
tab_df = contrast.loc[:, ("pct_diff", slice(None), slice(None), slice(None), keep_cols)]
tab_df = tab_df.droplevel([0], "columns")

util_df = contrast.loc[:, ("diff", slice(None), slice(None), slice(None), "utilization")]
util_df = util_df.droplevel([0], "columns")
tab_df = tab_df.merge(util_df, how="inner", left_index=True, right_index=True)

tab_df = tab_df.stack(["Vehicles per Fleet", "valet_set", "shar_priv", "variable"])
tab_df.name = "value"
tab_df = tab_df.reset_index()
tab_df = tab_df.groupby(["variable", "Vehicles per Fleet", "valet_set", "shar_priv"])["value"].describe(percentiles=[0.05, 0.5, 0.95])
tab_df = tab_df.loc[:, ["50%", "5%", "95%"]].round(1).astype(str)
tab_df["cell_text"] = tab_df["50%"] + "[" + tab_df["5%"] + "," + tab_df["95%"]+ "]"
tab_df["cell_text"].unstack(["Vehicles per Fleet", "valet_set", "shar_priv"])

### Table: changes vs Private, No Valet

For each metric, we show percent change relative to the `Private, No Valet` baseline by vocation-pair and vehicle scenario, then report the distribution:
- Median with 5th and 95th percentiles as `[p5, p95]`.
- Metrics: LCOC (`$/kWh`), capacity per vehicle (`kW/veh`), upfront vs continuing cost split, and the absolute difference in utilization.

Interpretation: Negative values indicate savings (lower cost or capacity) relative to the baseline.

Build another table, this time using "Shared, No Valet" as the baseline.

In [ ]:
from_shared = deepcopy(precursor)
for val_type, n_vehs, share, valet in from_shared.columns:
    from_shared[("diff", n_vehs, share, valet)] = from_shared[("orig", n_vehs, share, valet)] - from_shared[("orig", n_vehs, "shared", "no_valet")]
    from_shared[("pct_diff", n_vehs, share, valet)] = from_shared[("diff", n_vehs, share, valet)] / from_shared[("orig", n_vehs, "shared", "no_valet")] * 100
    from_shared.loc[from_shared[("diff", n_vehs, share, valet)] == 0, ("pct_diff", n_vehs, share, valet)] = 0

from_shared = from_shared.unstack("variable")

In [ ]:
keep_cols = ["cost_dollars_per_kwh", "capacity_kw_per_veh", "cost_upfront_per_kwh", "cost_continue_per_kwh"]
tab_df = from_shared.loc[:, ("pct_diff", slice(None), slice(None), slice(None), keep_cols)]
tab_df = tab_df.droplevel([0], "columns")

util_df = from_shared.loc[:, ("diff", slice(None), slice(None), slice(None), "utilization")]
util_df = util_df.droplevel([0], "columns")
tab_df = tab_df.merge(util_df, how="inner", left_index=True, right_index=True)

tab_df = tab_df.stack(["Vehicles per Fleet", "valet_set", "shar_priv", "variable"])
tab_df.name = "value"
tab_df = tab_df.reset_index()
tab_df = tab_df.groupby(["variable", "Vehicles per Fleet", "valet_set", "shar_priv"])["value"].describe(percentiles=[0.05, 0.5, 0.95])
tab_df = tab_df.loc[:, ["50%", "5%", "95%"]].round(1).astype(str)
tab_df["cell_text"] = tab_df["50%"] + "[" + tab_df["5%"] + "," + tab_df["95%"]+ "]"
tab_df["cell_text"].unstack(["Vehicles per Fleet", "valet_set", "shar_priv"])

### Table: changes vs Shared, No Valet

We repeat the summary using `Shared, No Valet` as the baseline to isolate the effect of valet staffing within shared depots, separating sharing benefits from valet-specific effects.

## Attempt graphical description

In [ ]:
same_voc = contrast.index.get_level_values("voc_category_i").to_series().values == contrast.index.get_level_values("voc_category_j").to_series().values
col_name = ["capacity_kw_per_veh", "cost_dollars_per_kwh"]
same_voc = contrast.loc[same_voc, ("pct_diff", slice(None), "shared", "no_valet", col_name)]
same_voc.describe(percentiles=[0.05, 0.5, 0.95])

In [ ]:
col_name = ["capacity_kw_per_veh", "cost_dollars_per_kwh"]
plot_df = contrast.loc[:, ("pct_diff", slice(None), slice(None), slice(None), col_name)]
#plug_loss_df = contrast.loc[:, ("diff", slice(None), slice(None), slice(None), ["n_plugs_high", "n_plugs_med"])]
#plot_df = pd.concat([plot_df, plug_loss_df], axis=1)
plot_df = plot_df.droplevel([0], "columns")
plot_df = plot_df.stack(["Vehicles per Fleet", "valet_set", "shar_priv"]).reset_index()
plot_df["savings_dollars_per_kwh"] = -plot_df["cost_dollars_per_kwh"]
plot_df["reduction_kw_per_veh"] = -plot_df["capacity_kw_per_veh"]
#plot_df["Dropped Higher-Power Charger"] = np.logical_or(plot_df["n_plugs_high"] < 0, plot_df["n_plugs_med"] < 0)
sing_voc_col = "Sharing fleets have\nsame vocation"
plot_df[sing_voc_col] = (plot_df["voc_category_i"] == plot_df["voc_category_j"]) & (plot_df["shar_priv"] == "shared")
plot_df[sing_voc_col] = pd.Categorical(plot_df[sing_voc_col], categories=[True, False], ordered=True) # "Fake 1", "Fake 2"
plot_df["Strategy"] = build_strategy_labels(plot_df["valet_set"], plot_df["shar_priv"])
plot_df = plot_df.sort_values(sing_voc_col)

# plt.figure(figsize=default_fig_dims_inches)
markers = {
    True: MarkerStyle('>'),
    False: MarkerStyle('o'),
    "Fake 1": MarkerStyle('<'),
    "Fake 2": MarkerStyle('.'),
}
g = sns.relplot(
    data=plot_df,
    x="reduction_kw_per_veh",
    y="cost_dollars_per_kwh",
    hue="Strategy",
    style=sing_voc_col,
    col="Vehicles per Fleet",
    kind="scatter",
    markers=markers,
    edgecolor="black",
    linewidth=1.25,
    alpha=0.7,
    palette="colorblind",
)

for ax in g.axes.flat:
    ax.xaxis.set_major_formatter(PercentFormatter(xmax=100, decimals=0))
    ax.yaxis.set_major_formatter(PercentFormatter(xmax=100, decimals=0))
    ax.set_xlabel("Reduction in Installed Capacity [kW/veh]")
    ax.set_ylabel("Change in Levelized Cost [$/kWh]")
    ax.hlines(y=0, xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], linestyles="dashed", colors="gray")
    ax.vlines(x=0, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], linestyles="dashed", colors="gray")
#sns.move_legend(g, loc="upper center", bbox_to_anchor=(0.7, 0.9), ncols=1) #bbox_to_anchor=(0.4, 0), ncols=2)

catalog.save("cost_reduce_vs_capacity_savings", g.figure)
g.figure

### Figure: Capacity reduction vs LCOC change

- X-axis: percent reduction in installed capacity per vehicle (to the right is better).
- Y-axis: percent change in levelized cost per kWh (down is better).
- Markers indicate whether fleets in a pair share the same vocation; colors denote strategy.

Quadrants make trade-offs explicit: bottom-right means both lower cost and lower capacity installed.

In [ ]:
plug_cols = ["n_plugs_low", "n_plugs_med", "n_plugs_high"]
plugs = contrast.loc[:, ("orig", slice(None), slice(None), slice(None), plug_cols)]
plugs = plugs.droplevel([0], "columns")
plugs = plugs.stack(["Vehicles per Fleet", "valet_set", "shar_priv"])
plugs

In [ ]:
mean_plugs = plugs.groupby(["Vehicles per Fleet", "valet_set", "shar_priv"]).mean()
#mean_plugs = mean_plugs.div(mean_plugs.sum(axis=1), axis=0)
mean_plugs = mean_plugs.melt(ignore_index=False, var_name="Plug Power Level", value_name="plug_share")
mean_plugs["Plug Power Level"] = mean_plugs["Plug Power Level"].str.extract(r"(?<=n_plugs_)(.+)")
mean_plugs["Plug Power Level"] = mean_plugs["Plug Power Level"].str.capitalize()
mean_plugs["Plug Power Level"] = pd.Categorical(mean_plugs["Plug Power Level"], categories=["Low", "Med", "High"], ordered=True)
mean_plugs = mean_plugs.reset_index()
mean_plugs["Strategy"] = build_strategy_labels(mean_plugs["valet_set"], mean_plugs["shar_priv"], sep=",\n")
mean_plugs = mean_plugs.sort_values(["Vehicles per Fleet", "Strategy", "Plug Power Level"], ascending=True)
mean_plugs["plug_cum"] = mean_plugs.groupby(["Vehicles per Fleet", "valet_set", "shar_priv"])["plug_share"].cumsum()
mean_plugs["Plug Power Level"] = mean_plugs["Plug Power Level"].cat.reorder_categories(new_categories=list(reversed(mean_plugs["Plug Power Level"].cat.categories)), ordered=True)

In [ ]:
mean_plugs.loc[mean_plugs["valet_set"] == "no_valet", :]

In [ ]:
#plt.figure(figsize=default_fig_dims_inches)
g = sns.catplot(
    data=mean_plugs.iloc[::-1],
    x="Strategy",
    y="plug_cum",
    hue="Plug Power Level",
    dodge=False,
    palette="YlOrBr_r",
    kind="bar",
    col="Vehicles per Fleet",
    #row="Has High Plugs",
    margin_titles=True,
)
for i, ax in enumerate(g.axes.flat):
    ax.set_ylabel("Mean Number of Plugs per Depot")
    ax.tick_params(axis='x', labelrotation=45)
    # ax.text(0.02, 1.05, f'({chr(97 + i)})', transform=ax.transAxes,
    #     fontsize=16, fontweight="bold", ha='left', va='top')
sns.move_legend(g, loc="upper center", bbox_to_anchor=(0.44, -0.05), ncols=3)

catalog.save("plug_contribs", g.figure)
g.figure

### Figure: Plug mix by strategy

Stacked bars show the mean number of plugs per depot by power level (Low/Med/High), comparing strategies and fleet sizes.

In [ ]:
cost_cols = ["cost_demand_dollars", "cost_energy_dollars", "cost_plugs_dollars", "cost_valet_dollars", "energy_delivered_kwh"]
costs = contrast.loc[:, ("orig", slice(None), slice(None), slice(None), cost_cols)]
costs = costs.droplevel([0], "columns")
costs = costs.stack(["Vehicles per Fleet", "valet_set", "shar_priv"])
abs_cols = list(costs.columns)
for col in abs_cols:
    costs[f"{col}_per_kwh"] = costs[col] / costs["energy_delivered_kwh"]
costs = costs.drop(abs_cols + ["energy_delivered_kwh_per_kwh"], axis=1)

In [ ]:
mean_costs = costs.groupby(["Vehicles per Fleet", "valet_set", "shar_priv"]).mean()
#mean_costs = mean_costs.div(mean_costs.sum(axis=1), axis=0)
mean_costs = mean_costs.melt(ignore_index=False, var_name="Cost Type", value_name="cost_share")
mean_costs["Cost Type"] = mean_costs["Cost Type"].str.extract(r"(?<=cost_)(.+)(?=_dollars_per_kwh)")
mean_costs["Cost Type"] = mean_costs["Cost Type"].str.capitalize()
mean_costs["Cost Type"] = pd.Categorical(mean_costs["Cost Type"], categories=["Energy", "Demand", "Plugs", "Valet"], ordered=True)
mean_costs = mean_costs.reset_index()
mean_costs["Strategy"] = build_strategy_labels(mean_costs["valet_set"], mean_costs["shar_priv"], sep=",\n")
mean_costs = mean_costs.sort_values(["Vehicles per Fleet", "Strategy", "Cost Type"], ascending=True)
mean_costs["cost_cum"] = mean_costs.groupby(["Vehicles per Fleet", "valet_set", "shar_priv"])["cost_share"].cumsum()
mean_costs["Cost Type"] = mean_costs["Cost Type"].cat.reorder_categories(new_categories=list(reversed(mean_costs["Cost Type"].cat.categories)), ordered=True)

In [ ]:
#plt.figure(figsize=default_fig_dims_inches)
g = sns.catplot(
    data=mean_costs.iloc[::-1],
    x="Strategy",
    y="cost_cum",
    hue="Cost Type",
    dodge=False,
    palette="husl",
    kind="bar",
    col="Vehicles per Fleet",
)
# ref_df = mean_costs.set_index(["valet_set", "shar_priv", "Cost Type"])
# ref_val = ref_df.loc[("no_valet", "private", "Plugs"), "cost_cum"]

def add_hline(data: pd.DataFrame, color=None):
    ref_df = data.set_index(["valet_set", "shar_priv", "Cost Type"])
    ref_val = ref_df.loc[("no_valet", "private", "Plugs"), "cost_cum"]
    ax = plt.gca()
    ax.hlines(y=ref_val, xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], linestyles="--", colors="gray")

g.map_dataframe(add_hline)
for ax in g.axes.flat:
    ax.set_ylabel("Cost Contribution [$/kWh]")
    ax.tick_params(axis='x', labelrotation=45)
    ax.yaxis.set_major_formatter('${x:1.2f}')
sns.move_legend(g, loc="upper center", bbox_to_anchor=(0.44, -0.05), ncols=4)

catalog.save("cost_contribs", g.figure)
g.figure

### Figure: Cost contributions to LCOC

Bars stack per-kWh contributions from Energy, Demand, Plugs (upfront), and Valet. The dashed line references the `Plugs` contribution in the `Private, No Valet` baseline.

In [ ]:
util = contrast.loc[:, ("orig", slice(None), slice(None), slice(None), "utilization")]
util = util.droplevel([0], "columns")
util = util.stack(["Vehicles per Fleet", "valet_set", "shar_priv"])
util = util.reset_index()
util["Strategy"] = build_strategy_labels(util["valet_set"], util["shar_priv"])

plt.figure(figsize=default_fig_dims_inches)
g = sns.displot(
    data=util,
    x="utilization",
    hue="Strategy",
    kind="ecdf",
    col="Vehicles per Fleet",
    palette="colorblind",
)

for ax in g.axes.flat:
    ax.xaxis.set_major_formatter(PercentFormatter(xmax=1))
    ax.yaxis.set_major_formatter(PercentFormatter(xmax=1))
    ax.set_xlabel("Charger Utilization")
    ax.set_ylabel("Proportion of Scenarios")
sns.move_legend(g, loc="upper center", bbox_to_anchor=(0.44, 0), ncols=2)

catalog.save("utilization_dists", g.figure)
g.figure

### Figure: Charger utilization distributions

The ECDF shows the distribution of average utilization across scenarios for each strategy and fleet size.

Interpretation: Curves further right indicate generally higher utilization (i.e., more of the capacity is used over time).

## Explore how vocation pairs influence results

In [ ]:
def build_heat_array(data: pd.DataFrame, color=None, sum_col: str = None, **kwargs) -> Axes:
    voc_cols = ("voc_category_i", "voc_category_j")
    heat = data.groupby(list(voc_cols))[sum_col].agg(["mean", "std"])
    heat = heat.reset_index()
    heat[list(voc_cols)] = heat[list(voc_cols)].transform(lambda x: x.str.replace("_", " ").str.title())
    heat = heat.set_index(list(voc_cols))
    heat["mean_annot"] = heat["mean"].round(1).astype(str)
    mean_pairs = build_pairwise_array(heat["mean"], voc_cols)
    ax = sns.heatmap(
        mean_pairs,
        # annot=mean_pairs,
        **kwargs,
    ) 
    ax.grid(visible=False)
    ax.set_xlabel("Potential Partner Vocation")
    ax.set_ylabel("Your Vocation")
    return ax

### First, investigate how vocation pairs influence LCOC

In [ ]:
gridder = contrast.loc[:, ("pct_diff", slice(None), slice(None), slice(None), "cost_dollars_per_kwh")]
gridder = gridder.droplevel([0], "columns")
inter_cols = ["Vehicles per Fleet", "valet_set", "shar_priv"]
gridder = gridder.stack(inter_cols).reset_index(inter_cols, drop=False)
gridder["Strategy"] = build_strategy_labels(gridder["valet_set"], gridder["shar_priv"])
gridder = gridder.loc[gridder["shar_priv"]=="shared", :]
gridder["Strategy"] = gridder["Strategy"].cat.remove_unused_categories()
gridder

In [ ]:
# Create a common color map and normalization
map_vals = gridder['cost_dollars_per_kwh']
vmin, vmax = map_vals.min(), map_vals.max()
cmap = sns.color_palette("light:b_r", as_cmap=True)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

# Create a colorbar with a single scalar mappable
fig = plt.figure()
cbar = fig.colorbar(mappable=sm, ax=plt.gca(), orientation='vertical', format=FuncFormatter(get_cbar_pct_labels), label="Avg. Pct. Change in Cost [$/kWh]")


In [ ]:
g = sns.FacetGrid(data=gridder, col="Strategy", row="Vehicles per Fleet", margin_titles=True)
g.map_dataframe(
    build_heat_array,
    sum_col="cost_dollars_per_kwh",
    square=True,
    cbar=False,
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
)
g.set_titles(col_template="Strategy =\n{col_name}", row_template="Vehicles per\nFleet = {row_name}")
g.set_axis_labels(x_var="Potential Partner\nVocation", y_var="Your Vocation")
#g.tick_params(axis="x", rotation=45)
#g.figure.set_size_inches(w=12, h=8)
catalog.save("sharing_cost_by_pair", g.figure)
g.figure

### Figure: LCOC impact by vocation pair

Each cell shows the average percent change in `$/kWh` for sharing strategies, grouped by your vocation (rows) and partner vocation (columns).

Interpretation: Cooler colors (negative) indicate cost savings from pairing; patterns reveal complementary duty profiles across vocations.

In [ ]:
g = sns.relplot(
    data=melted_comp,
    x="effective_size",
    y="capacity_kw_per_veh",
    hue="Vocation",
    kind="line",
    estimator="mean",
    errorbar=None,
)
for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(PercentFormatter(xmax=100, decimals=0))
    ax.set_ylabel("Reduction in Installed Capacity [kW/veh]")
    ax.set_xlabel("Fleet Size [vehs]")
    ax.set_ylim(bottom=0, top=ax.get_ylim()[1])
    #ax.set_xlim(left=0, right=ax.get_xlim()[1])

#sns.move_legend(g, loc="upper center", bbox_to_anchor=(0.4, 0), ncols=2)

catalog.save("capacity_savings_by_fleet_size", g.figure)
g.figure
    

### Effective fleet size comparison (same-vocation pairs)

We compare capacity reductions for private vs shared (with valet) scenarios by an "effective size" (5/10/15/30 vehicles).

## Plot load-duration curves

In [ ]:
power = profs_df.reset_index()
grp_cols = ["Vehicles per Fleet", "voc_category_i", "voc_category_j", "valet_set", "shar_priv", "veh_scen_id"]

depot_power_ser = power.groupby(grp_cols + ["time"], observed=True)["avg_power_kw"].sum()
plot_df = depot_power_ser.reset_index()
plot_df = plot_df.sort_values(grp_cols + ["avg_power_kw"], ascending=False)

def norm(x: pd.Series) -> pd.Series:
    return np.arange(1, x.count() + 1) / (x.count() + 1) * 100

plot_df["util_rate_pct"] = plot_df.groupby(grp_cols, observed=True)["avg_power_kw"].transform(norm)
plot_df = plot_df.reset_index(drop=True)
plot_df = plot_df.sort_values(grp_cols)
plot_df["Strategy"] = build_strategy_labels(plot_df["valet_set"], plot_df["shar_priv"])

In [ ]:
g = sns.relplot(
    plot_df,
    x="util_rate_pct",
    y="avg_power_kw",
    hue="Strategy",
    col="Vehicles per Fleet",
    kind="line",
    errorbar="ci",
    palette="colorblind",
)

for ax in g.axes.flat:
    ax.set_xlabel("Capacity Utilization Rate")
    ax.xaxis.set_major_formatter(PercentFormatter(xmax=100))
    ax.set_ylabel("Power [kW]")
sns.move_legend(g, loc="upper center", bbox_to_anchor=(0.44, 0), ncols=2)

catalog.save("load_duration_by_strategy", g.figure)
g.figure

### Figure: Load-duration curves by strategy

We aggregate depot power over time and convert to a load-duration curve (power vs percentile of time).